In [39]:
!pip install geocoder
!pip install folium
!pip install pandas
!pip install geopy
!pip install bs4
!pip install matplotlib
!pip install sklearn
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [9]:
data = requests.get("https://metroatlantasuburbs.com/communities/").text

In [10]:
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("a", class_="fl-button"):
    neighborhoodList.append(row.text.strip())

In [11]:
neighborhoodList1 = neighborhoodList[:-1]

In [12]:
df_final=pd.DataFrame({'Neighborhood':neighborhoodList1})
df_final

,Neighborhood
0,ALPHARETTA
1,ACWORTH
2,BROOKHAVEN
3,BUCKHEAD
4,BUFORD
5,CANTON
6,CLARKSTON
7,CHAMBLEE
8,CUMMING
9,DECATUR (CITY)


In [13]:
# to get lenght of dataframe
df_final.shape

(30, 1)

Get coordinates of neighborhood in Atlanta,Ga

In [14]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Atlanta,Georgia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [15]:
lat_lng = [get_latlng(neighborhood) for neighborhood in df_final['Neighborhood'].tolist()]

In [11]:
lat_lng

[[34.07540000000006, -84.29454999999996],
 [33.74831000000006, -84.39110999999997],
 [33.86024188880596, -84.34390227668973],
 [33.86434000000003, -84.42104999999998],
 [33.8674751390884, -84.30133506654134],
 [33.782170000000065, -84.36802999999998],
 [33.80908000000005, -84.23961999999995],
 [33.89291900000006, -84.30878899999999],
 [33.711105444543264, -84.42890544449364],
 [33.74831000000006, -84.39110999999997],
 [33.78809000000007, -84.32662399999998],
 [33.93384296274751, -84.30224858023277],
 [33.74831000000006, -84.39110999999997],
 [33.990350308432284, -84.34324539690766],
 [33.76601003751439, -84.40883479753599],
 [33.7665743599851, -84.43431923447164],
 [33.80857400000008, -84.41925799999996],
 [33.80773504674795, -84.37235492019559],
 [33.81877000000003, -84.58009999999996],
 [33.77392700000007, -84.46757899999994],
 [33.74296561879687, -84.41489998846251],
 [33.977966908157654, -84.35124958860627],
 [33.926694000000055, -84.37804699999998],
 [33.791980000000024, -84.26458

In [16]:
df = pd.DataFrame(lat_lng , columns=['Latitude','Longitude'])
df_final['Latitude'] = df['Latitude']
df_final['Longitude'] = df['Longitude']
#df_final.drop('Lonitude',axis=1,inplace=True)
df_final

,Neighborhood,Latitude,Longitude
0,ALPHARETTA,34.075400,-84.294550
1,ACWORTH,33.748310,-84.391110
2,BROOKHAVEN,33.860242,-84.343902
3,BUCKHEAD,33.864340,-84.421050
4,BUFORD,33.867475,-84.301335
5,CANTON,33.782170,-84.368030
6,CLARKSTON,33.809080,-84.239620
7,CHAMBLEE,33.892919,-84.308789
8,CUMMING,33.711105,-84.428905
9,DECATUR (CITY),33.748310,-84.391110


        Create a map of Atlanta with neighborhoods superimposed on top

In [18]:
#Manually created this file from website http://zipatlas.com/us/ga/city-comparison/percentage-indian-population.htm
#my_file = project.get_file("file2.csv")
df_pop = pd.read_csv("file2.csv")
df_pop.fillna('',inplace=True)
df_pop['%Indian(Asian)'] = df_pop['%Indian(Asian)'].astype('str') + '%'
df_pop

,Neighborhood,Population,%Indian(Asian)
0,ALPHARETTA,"1,22,873",2.09%
1,ACWORTH,"72,182",0.48%
2,BROOKHAVEN,"36,780",0.94%
3,BUCKHEAD,"1,834",0.001%
4,BUFORD,"48,297",0.28%
5,CANTON,"42,399",0.09%
6,CLARKSTON,"24,374",1.73%
7,CHAMBLEE,,%
8,CUMMING,"75,140",0.14%
9,DECATUR (CITY),"25,137",0.71%


Intial Analysis:
Lawrenceville and Alpharetta has high population but they have less %indians as compared to others,so we should not consider them as our first choice.
Duluth,Dunwoody has quite good %indians and low population, so we can consider this as one of our choice.
Decatur(Emory) has highest %indians with regard to total population of place.So we should consider this as our first choice


In [41]:
# get the coordinates of Atlanta
address = 'Atlanta,Georgia'

geolocator = Nominatim(user_agent="foursquare-agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Atlanta,Georgia {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_atl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_atl)  
    
map_atl

The geograpical coordinate of Atlanta,Georgia 33.7489924, -84.3902644.


    Use the Foursquare API to explore the Neighborhoods

In [20]:
CLIENT_ID = 'XI5WB3EYGHFMHLBWHDDYGTWJGODRMZW53UL2UEAV4V3I5B1K' # your Foursquare ID
CLIENT_SECRET = 'ORWT4HP2BKOMNQWQZPEVJRZVT0MXTQZ3FBXBUKGCMULYX1NR' # your Foursquare Secret
ACCESS_TOKEN = '5KDPEMMOBD2C5MCS23I1EAVXQKJRQMM45H1YSTH0PJUJGCEQ' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    LIMIT = 100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
atlanta_venues_df = getNearbyVenues(names=df_final['Neighborhood'],
                                   latitudes=df_final['Latitude'],
                                   longitudes=df_final['Longitude']
                                  )

In [32]:
atlanta_venues_df

,Neighborhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALPHARETTA,34.075400,-84.294550,Smokejack BBQ,34.074873,-84.294734,BBQ Joint
1,ALPHARETTA,34.075400,-84.294550,Alpharetta Food Truck Alley,34.075192,-84.295093,Food Truck
2,ALPHARETTA,34.075400,-84.294550,Alpharetta Square,34.075586,-84.295196,Plaza
3,ALPHARETTA,34.075400,-84.294550,made kitchen & cocktails,34.074449,-84.296886,Tapas Restaurant
4,ALPHARETTA,34.075400,-84.294550,Crust Pasta & Pizzeria,34.071709,-84.296617,Italian Restaurant
5,ALPHARETTA,34.075400,-84.294550,Flatlands Bourbon and Bayou,34.077032,-84.295022,Cajun / Creole Restaurant
6,ALPHARETTA,34.075400,-84.294550,The Nest Cafe,34.077043,-84.296538,Café
7,ALPHARETTA,34.075400,-84.294550,Maple Street Street Biscuit Co.,34.075884,-84.297218,Breakfast Spot
8,ALPHARETTA,34.075400,-84.294550,Alpharetta Farmer's Market,34.075784,-84.295113,Flea Market
9,ALPHARETTA,34.075400,-84.294550,Collet French Pastry & Cafe,34.070248,-84.293311,French Restaurant


In [21]:
atlanta_venues_df.groupby(['Neighborhood']).count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ACWORTH,100,100,100,100,100,100
ALPHARETTA,100,100,100,100,100,100
BROOKHAVEN,100,100,100,100,100,100
BUCKHEAD,31,31,31,31,31,31
BUFORD,73,73,73,73,73,73
CANTON,100,100,100,100,100,100
CHAMBLEE,100,100,100,100,100,100
CLARKSTON,33,33,33,33,33,33
CUMMING,28,28,28,28,28,28


In [25]:
print("There are {} unique categories".format(len(atlanta_venues_df['Venue Category'].unique())))

There are 274 unique categories


In [22]:
atlanta_onehot = pd.get_dummies(atlanta_venues_df[['Venue Category']],prefix="",prefix_sep="")
atlanta_onehot['Neighborhood'] = atlanta_venues_df['Neighborhood']
#move neighborhood column as first column
fixed_columns = [atlanta_onehot.columns[-1]] + list(atlanta_onehot.columns[:-1])
atlanta_onehot = atlanta_onehot[fixed_columns]
atlanta_onehot.head()
atlanta_onehot.shape



(2181, 274)

In [24]:
atlanta_grp=atlanta_onehot.groupby(['Neighborhood']).mean().reset_index()
#atlanta_grp.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
atlanta_grp

,Neighborhood,Zoo Exhibit,Accessories Store,Adult Boutique,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cemetery,Chinese Restaurant,Church,City,Clothing Store,Cocktail Bar,Coffee Shop,College & University,College Academic Building,College Basketball Court,College Bookstore,College Library,College Rec Center,College Theater,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Daycare,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Flea Market,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Laundromat,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Nature Preserve,New American Restaurant,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Optical Shop,Other Great Outdoors,Outdoor Sculpture,Outlet Store,Paper / Office Supplies Store,Park,Parking,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Rafting,Recording Studio,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Roller Rink,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shipping Store,Shoe Repair,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tanning Salon,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Trail,Train Station,Tunnel,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio
0,ACWORTH,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00

In [25]:
len(atlanta_grp[atlanta_grp['Indian Restaurant'] > 0])

9

In [29]:
atlanta_new = atlanta_grp[['Neighborhood','Indian Restaurant']]
atlanta_new


,Neighborhood,Indian Restaurant
0,ACWORTH,0.000000
1,ALPHARETTA,0.010000
2,BROOKHAVEN,0.000000
3,BUCKHEAD,0.000000
4,BUFORD,0.027397
5,CANTON,0.010000
6,CHAMBLEE,0.010000
7,CLARKSTON,0.030303
8,CUMMING,0.000000
9,DECATUR (CITY),0.000000


cluster Neighbourhoods

In [30]:
cluster_num=3
kl_clustering = atlanta_new.drop(["Neighborhood"], axis=1)
k_means = KMeans(n_clusters=cluster_num,random_state=0).fit(kl_clustering)
labels = k_means.labels_
labels

array([0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0])

In [34]:
atlanta_new = atlanta_new.copy()
atlanta_new['cluster_labels']= labels
atlanta_new

,Neighborhood,Indian Restaurant,cluster_labels
0,ACWORTH,0.000000,0
1,ALPHARETTA,0.010000,0
2,BROOKHAVEN,0.000000,0
3,BUCKHEAD,0.000000,0
4,BUFORD,0.027397,2
5,CANTON,0.010000,0
6,CHAMBLEE,0.010000,0
7,CLARKSTON,0.030303,2
8,CUMMING,0.000000,0
9,DECATUR (CITY),0.000000,0


Merging  indian Restaurant with indian demographic

In [35]:
atlanta_merged = pd.merge(atlanta_new,df_final,on='Neighborhood',how='inner')

In [36]:
atlanta_merged

,Neighborhood,Indian Restaurant,cluster_labels,Latitude,Longitude
0,ACWORTH,0.000000,0,33.748310,-84.391110
1,ALPHARETTA,0.010000,0,34.075400,-84.294550
2,BROOKHAVEN,0.000000,0,33.860242,-84.343902
3,BUCKHEAD,0.000000,0,33.864340,-84.421050
4,BUFORD,0.027397,2,33.867475,-84.301335
5,CANTON,0.010000,0,33.782170,-84.368030
6,CHAMBLEE,0.010000,0,33.892919,-84.308789
7,CLARKSTON,0.030303,2,33.809080,-84.239620
8,CUMMING,0.000000,0,33.711105,-84.428905
9,DECATUR (CITY),0.000000,0,33.748310,-84.391110


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(cluster_num)
ys = [i+x+(i*x)**2 for i in range(cluster_num)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(atlanta_merged['Latitude'], atlanta_merged['Longitude'], atlanta_merged['Neighborhood'], atlanta_merged['cluster_labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
Here, we can see best suburbs/neighbourhood around Atlanta city,Georgia.Each neighbourhood has various venues.In this project, we are particularly interested in opening an Indian Restarurant within 2000 m of Atlanta.In the map shown above, we can clearly see 3different types of cluster shown in different colors.
The most prominent one is red cluster with value 0 as it includes places where there are no indian restaurant.The places where we can see indian restaurant already is cluster 2.So, these places are not good choice to open an indian restaurant.There is a place Scotdale having cluster value 1(purple color)
does not seem to be good choice either as it is least  populated with less indian population also.The best place to open an indian restaurant is place where there is maximum indian population and less number of indian
restaurants already present.We can say,cluster 0 is best option.Among all the places having cluster 0, Decatur(Emory) and cumming would be good choice as both places have considerable large amount of indian population with respect to total population.
